# PyCBC Practice
Elizabeth Kapelevich

In [1]:
from pycbc.waveform import get_td_waveform      # imports a time domain waveform module
import pycbc.noise
from pycbc.detector import Detector
import matplotlib.pyplot as plt
import matplotlib
from pycbc.psd import aLIGOZeroDetHighPower
import numpy as np

ModuleNotFoundError: No module named 'pycbc'

In [ ]:
time = 32                   # length of the signal
rate = 4096                 # sampling rate of the signal (one second of data has 4096 points)
f_low = 20                  # low-frequency threshold
tlen = rate * time          # length of the wave array is the product of total duration and sampling rate

In [ ]:
sp, sc = get_td_waveform(approximant="SpinTaylorT4", mass1=5, mass2=5,
                        f_lower=f_low, delta_t=1.0/rate, inclination=0, phase_order=7,
                         amplitude_order=0, distance=75)

In [ ]:
sp.resize(tlen)
sc.resize(tlen)

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.figure(figsize=(12, 7))
plt.plot(sp.sample_times, sp)
plt.xlabel("Time (sec)")
plt.ylabel("Strain")
plt.show()

In [ ]:
delta_f = 1/time
flen = int(tlen/2 + 1)

In [ ]:
psd = aLIGOZeroDetHighPower(flen, delta_f, f_low)    # f_low is where it starts, length is flen, 
                                                     # and delta_f is frequency sampling rate

In [ ]:
np.sqrt(psd.at_frequency(25))     # sensitivity of LIGO at given frequency

In [ ]:
frequency = np.linspace(20, 2000, 10000)
psd_vals = []
for f in frequency:
    psd_vals.append(psd.at_frequency(f))
psd_vals = np.array(psd_vals)

In [ ]:
plt.loglog(frequency, psd_vals)  # makes both axes logs

In [ ]:
# generating noise instance

delta_t = 1/rate
ts = pycbc.noise.noise_from_psd(tlen, delta_t, psd, seed = 100)          # pycbc noise module

In [ ]:
plt.plot(ts.sample_times, ts)

In [ ]:
# converting noise spectrum into frequency series

fs = ts.to_frequencyseries()
keep = fs.sample_frequencies.data > 20

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.figure(figsize=(12,7))
plt.loglog((fs.sample_frequencies)[keep], np.abs(fs.data)[keep]**2, alpha=0.4, label="actual noise")
plt.loglog(frequency, psd_vals, label="psd from which noise is generated")
plt.legend()

In [ ]:
# embedding signal within noise

ra = 1.2      # right ascension (radians)
dec = 1.7     # declination
psi = 0.1     # polarization angle
t = 1343586137

dh = Detector("H1")
dl = Detector("L1")
dv = Detector("V1")

In [ ]:
fph, fch = dh.antenna_pattern(ra, dec, psi, t)
fpl, fcl = dl.antenna_pattern(ra, dec, psi, t)
ht = (fph * sp) + (fch * sc)  # created all orthogonal transofmrations
hl = (fpl * sp) + (fcl * sc)

In [ ]:
sp.start_time = sc.start_time = t

In [ ]:
plt.rcParams.update({'font.size': 18})
plt.figure(figsize=(12, 7))
plt.plot(sp.sample_times, sp)   # set it to the start time of the wave, no longer relative to 0!
plt.xlabel("Time (sec)")
plt.ylabel("Strain")
plt.show()

In [ ]:
# directly creating h(t)

ht_hanford = dh.project_wave(sp, sc, ra, dec, psi)
ht_livingston = dl.project_wave(sp, sc, ra, dec, psi)

In [ ]:
ht_hanford.start_time

In [ ]:
ht_livingston.start_time

In [ ]:
time_delay = ht_livingston.start_time - ht_hanford.start_time

In [ ]:
time_delay.gpsNanoSeconds * 1e-6

In [ ]:
len(ht_hanford.data)

In [ ]:
len(ts)

In [ ]:
ht_hanford.resize(len(ts))

In [ ]:
len(ht_hanford)

In [ ]:
ts.start_time = ht_hanford.start_time.gpsSeconds + ht_hanford.start_time.gpsNanoSeconds * 1e-9
data_hanford = ht_hanford + ts

In [ ]:
# creating template using same parameters

hp, hc = get_td_waveform(approximant="SpinTaylorT4", mass1=5, mass2=5,
                        f_lower=f_low, delta_t=1.0/rate, inclination=0, phase_order=7,
                         amplitude_order=0, distance=75)

Use command to create `sp` as given.
Run a nested loop over masses between 1 to 10 solar masses and save the match between all of those points.
It should calculate the match between those masses in the template and the signal. Only change the masses in the template.
Plot a heat map. 
Chirp mass: $ m_c = \frac{(m_1m_2)^{3/5}}{(m_1+m_2)^{1/5}} $

Figure out zero-padding.